In [1]:
%load_ext autoreload
%autoreload 2

import deltas
from deltas.pipeline import data, classifier, evaluation
from deltas.model import downsample
from tqdm import tqdm

import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
# np.random.seed(10)


In [2]:

model = 'MNIST'

# dataset = datasets[7]  # change ind to select dataset to use
dfs = []
for i in tqdm(range(10)):
    # np.random.seed(random.randint)
    # np.random.seed(i)
    data_clf = data.get_real_dataset(
        'MNIST', _print=False, seed=i, scale=True, minority_id=[3], classes_remove=[4,5,6,7,8,9])
        # 'MNIST', _print=False, seed=i, scale=True, minority_id=[i])

    classifiers_dict = classifier.get_classifier(
        data_clf=data_clf,
        model=model,
        _plot=False,
        _print=False)
    data_clf['clf'] = classifiers_dict['Baseline']
    X = data_clf['data']['X']
    y = data_clf['data']['y']
    clf = data_clf['clf']
    # deltas_model = downsample.downsample_deltas(
    #     clf).fit(X, y, _print=True, _plot=True, max_trials=10000)
    # deltas_model = base.base_deltas(
    #     clf).fit(X, y, grid_search=True, _print=True, _plot=True)
    if False:
        param_grid = {
                    #   'alpha': [0, 0.1, 1, 10],
                    #   'grid_search': [True, False],
                    'method': ['supports-prop-update_mean', 'supports-prop-update_mean-margin_only']}
        grid_original = GridSearchCV(
            downsample.downsample_deltas(), param_grid, refit=True)
        grid_original.fit(X, y,
                        clf=clf,
                        _print=False,
                        _plot=False,
                        max_trials=10000,
                        parallel=True)
        deltas_model = grid_original.best_estimator_
        print(f'Best params: {grid_original.best_params_}')
    else:
        deltas_model = downsample.downsample_deltas(clf).fit(X, y,
                                                             alpha=10,
                                                             _print=False,
                                                             _plot=False,
                                                             method='supports-prop-update_mean',
                                                             max_trials=10000,
                                                             parallel=True)

    if deltas_model.is_fit == True:
        classifiers_dict['Our Method'] = deltas_model
        scores_df = evaluation.eval_test(classifiers_dict,
                                         data_clf['data_test'], _print=True, _plot=False)
        dfs.append(scores_df)
    else:
        print('not fit deltas')

  0%|          | 0/10 [00:00<?, ?it/s]

30596 5842


/home/matt/projects/linear_confidence/deltas/utils/equations.py:145: RuntimeWarning: invalid value encountered in scalar divide
  inside_exp = 0.5*(np.square(((B*np.sqrt(N2))/(factor*R2)) - 2))
/home/matt/projects/linear_confidence/deltas/utils/equations.py:145: RuntimeWarning: invalid value encountered in scalar divide
  inside_exp = 0.5*(np.square(((B*np.sqrt(N2))/(factor*R2)) - 2))
/home/matt/projects/linear_confidence/deltas/utils/equations.py:145: RuntimeWarning: invalid value encountered in scalar divide
  inside_exp = 0.5*(np.square(((B*np.sqrt(N2))/(factor*R2)) - 2))
/home/matt/projects/linear_confidence/deltas/utils/equations.py:145: RuntimeWarning: invalid value encountered in scalar divide
  inside_exp = 0.5*(np.square(((B*np.sqrt(N2))/(factor*R2)) - 2))
/home/matt/projects/linear_confidence/deltas/utils/equations.py:145: RuntimeWarning: invalid value encountered in scalar divide
  inside_exp = 0.5*(np.square(((B*np.sqrt(N2))/(factor*R2)) - 2))
/home/matt/projects/linear_con

not fit deltas


 10%|█         | 1/10 [00:56<08:28, 56.51s/it]


KeyboardInterrupt: 

In [ ]:
print(len(dfs))

In [ ]:
df = pd.concat(dfs, axis=0)
mean = {}
std = {}
index = df.index.unique().to_list()
cols = df.columns.to_list()
for method in index:
    mean[method] = df.loc[method].mean().to_list()
    std[method] = df.loc[method].std().to_list()

mean_df = pd.DataFrame.from_dict(mean, orient='index', columns=cols)
std_df = pd.DataFrame.from_dict(std, orient='index', columns=cols)

m = mean_df.to_dict('list')
s = std_df.to_dict('list')
metrics = mean_df.columns.to_list()
methods = mean_df.index.to_list()
sf = 5
for metric in metrics:
    means = m[metric]
    sts = s[metric]
    mx = np.argmax(means)
    for i in range(len(means)):
        m_str = str(means[i])[1:sf]
        if i == mx:
            m_str = f"\\textbf{{{m_str}}}"
        s_str = str(sts[i])[1:sf-1]
        m[metric][i] = f'${m_str} \\pm {s_str}$'

method_map = {
    'Baseline': 'Baseline',
    'SMOTE': "SMOTE \cite{Chawla_2002_JAIR}",
    'Balanced Weights': 'BW',
    'BMR': 'BMR \cite{Bahnsen_2014_SIAM}',
    'Threshold': 'Thresh \cite{Sheng_2006_AAAI}',
    'Our Method': 'Our Method',
}
meths_new = []
for me in methods:
    meths_new.append(method_map[me])
m['Methods'] = meths_new
df = pd.DataFrame(m)  # .set_index('Methods')
meths = df.pop('Methods')
df.insert(0, 'Methods', meths)
latex_str = df.to_latex(index=False)

print('\\begin{tabular}{@{}lrrrr@{}}'+latex_str[22:])